<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [41]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [43]:
from sklearn import set_config
set_config(transform_output="pandas")

In [44]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["quantity"])
y = df["quantity"]

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.33, random_state=42, shuffle=False)

In [45]:
from sklearn.preprocessing import FunctionTransformer

def get_info_date(df):
    date = df["date"]
    day = date.apply(lambda x: x.split("/")[0])
    month = date.apply(lambda x: x.split("/")[1])
    year = date.apply(lambda x: "20" + x.split("/")[2])
    df["day"] = pd.Categorical(day)
    df["month"] = pd.Categorical(month)
    df["year"] = pd.Categorical(year)

    return df.drop(columns=["date"])

date_transformer = FunctionTransformer(get_info_date, validate=False)

In [46]:
categ_cols = ["city", "shop", "brand", "container", "capacity", "year", "month", "day"]
numeric_cols = ["lat", "long", "pop", "price"]

In [47]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

categ_numeric_transformer = ColumnTransformer(
    transformers=[
        ("categ", OneHotEncoder(sparse_output=False), categ_cols),
        ("numeric", MinMaxScaler(), numeric_cols)
    ],
)

preprocessor = Pipeline(
    steps=[
        ("date", date_transformer),
        ("categ_numeric", categ_numeric_transformer)
    ]
).set_output(transform="pandas")

dummy_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("dummy", DummyRegressor())
    ]
)

dummy_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('date',
                                  FunctionTransformer(func=<function get_info_date at 0x00000273D3687560>)),
                                 ('categ_numeric',
                                  ColumnTransformer(transformers=[('categ',
                                                                   OneHotEncoder(sparse_output=False),
                                                                   ['city',
                                                                    'shop',
                                                                    'brand',
                                                                    'container',
                                                                    'capacity',
                                                                    'year',
                                                                    'month',
                                                                    'day']),
                                                                  ('numeric',
                                                                   MinMaxScaler(),
                                                                   ['lat',
                                                                    'long',
                                                                    'pop',
                                                                    'price'])]))])),
                ('dummy', DummyRegressor())])

In [48]:
val_pred = dummy_pipeline.predict(X_val)
mean_absolute_error(y_val, val_pred)

ValueError: Found unknown categories ['2018', '2017'] in column 5 during transform

In [29]:
from xgboost import XGBRegressor

xgbregressor_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("xgb", XGBRegressor())
    ]
)

xgbregressor_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('date',
                                  FunctionTransformer(func=<function get_info_date at 0x00000273D202AAC0>)),
                                 ('categ_numeric',
                                  ColumnTransformer(transformers=[('categ',
                                                                   OneHotEncoder(sparse_output=False),
                                                                   ['city',
                                                                    'shop',
                                                                    'brand',
                                                                    'container',
                                                                    'capacity',
                                                                    'year',
                                                                    'month',
                                                                    'day']),
                                                                  ('numeric',
                                                                   MinMaxScaler(),
                                                                   ['lat',
                                                                    'long',
                                                                    'pop',
                                                                    'price'])]))]))...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [30]:
xgbregressor_pred = xgbregressor_pipeline.predict(X_val)
mean_absolute_error(xgbregressor_pred, y_val)

2415.825836629829

In [31]:
import joblib

joblib.dump(dummy_pipeline, "dummy_pipeline_LUCAS.pkl")
joblib.dump(xgbregressor_pipeline, "xgbregressor_pipeline_LUCAS.pkl")

['xgbregressor_pipeline_LUCAS.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [32]:
cols_post_preprocessing = xgbregressor_pipeline[0][1].get_feature_names_out()
cols_post_preprocessing

array(['categ__city_Athens', 'categ__city_Irakleion',
       'categ__city_Larisa', 'categ__city_Patra',
       'categ__city_Thessaloniki', 'categ__shop_shop_1',
       'categ__shop_shop_2', 'categ__shop_shop_3', 'categ__shop_shop_4',
       'categ__shop_shop_5', 'categ__shop_shop_6',
       'categ__brand_adult-cola', 'categ__brand_gazoza',
       'categ__brand_kinder-cola', 'categ__brand_lemon-boost',
       'categ__brand_orange-power', 'categ__container_can',
       'categ__container_glass', 'categ__container_plastic',
       'categ__capacity_1.5lt', 'categ__capacity_330ml',
       'categ__capacity_500ml', 'categ__year_2012', 'categ__year_2013',
       'categ__year_2014', 'categ__year_2015', 'categ__year_2016',
       'categ__year_2017', 'categ__year_2018', 'categ__month_01',
       'categ__month_02', 'categ__month_03', 'categ__month_04',
       'categ__month_05', 'categ__month_06', 'categ__month_07',
       'categ__month_08', 'categ__month_09', 'categ__month_10',
       'categ__month

In [33]:
cols_constraints = {
    col: -1 if col == "numeric__price" else 0 for col in cols_post_preprocessing
}
cols_constraints

{'categ__city_Athens': 0,
 'categ__city_Irakleion': 0,
 'categ__city_Larisa': 0,
 'categ__city_Patra': 0,
 'categ__city_Thessaloniki': 0,
 'categ__shop_shop_1': 0,
 'categ__shop_shop_2': 0,
 'categ__shop_shop_3': 0,
 'categ__shop_shop_4': 0,
 'categ__shop_shop_5': 0,
 'categ__shop_shop_6': 0,
 'categ__brand_adult-cola': 0,
 'categ__brand_gazoza': 0,
 'categ__brand_kinder-cola': 0,
 'categ__brand_lemon-boost': 0,
 'categ__brand_orange-power': 0,
 'categ__container_can': 0,
 'categ__container_glass': 0,
 'categ__container_plastic': 0,
 'categ__capacity_1.5lt': 0,
 'categ__capacity_330ml': 0,
 'categ__capacity_500ml': 0,
 'categ__year_2012': 0,
 'categ__year_2013': 0,
 'categ__year_2014': 0,
 'categ__year_2015': 0,
 'categ__year_2016': 0,
 'categ__year_2017': 0,
 'categ__year_2018': 0,
 'categ__month_01': 0,
 'categ__month_02': 0,
 'categ__month_03': 0,
 'categ__month_04': 0,
 'categ__month_05': 0,
 'categ__month_06': 0,
 'categ__month_07': 0,
 'categ__month_08': 0,
 'categ__month_09': 0,

In [34]:
# Inserte su código acá
params = {}
params["monotone_constraints"] = cols_constraints

# Le damos params a XGBRegressor como parámetro del fit
xgbregressor_pipeline[1].set_params(**params)
xgbregressor_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('date',
                                  FunctionTransformer(func=<function get_info_date at 0x00000273D202AAC0>)),
                                 ('categ_numeric',
                                  ColumnTransformer(transformers=[('categ',
                                                                   OneHotEncoder(sparse_output=False),
                                                                   ['city',
                                                                    'shop',
                                                                    'brand',
                                                                    'container',
                                                                    'capacity',
                                                                    'year',
                                                                    'month',
                                                                    'day']),
                                                                  ('numeric',
                                                                   MinMaxScaler(),
                                                                   ['lat',
                                                                    'long',
                                                                    'pop',
                                                                    'price'])]))]))...
                                                    'categ__day_29': 0,
                                                    'categ__day_30': 0,
                                                    'categ__day_31': 0,
                                                    'categ__month_01': 0,
                                                    'categ__month_02': 0,
                                                    'categ__month_03': 0,
                                                    'categ__month_04': 0,
                                                    'categ__month_05': 0,
                                                    'categ__month_06': 0,
                                                    'categ__month_07': 0,
                                                    'categ__month_08': 0,
                                                    'categ__month_09': 0,
                                                    'categ__month_10': 0, ...},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [35]:
monotone_val_pred = xgbregressor_pipeline.predict(X_val)
mean_absolute_error(monotone_val_pred, y_val)

2480.7012066783827

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [36]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

def objective(trial):
    # Inserte su código acá
    pass

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [37]:
#!pip install optuna-integration[xgboost]

In [38]:
# Inserte su código acá

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [39]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [40]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>